In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/03/01 22:41:18 WARN Utils: Your hostname, CDYVRCWLT614 resolves to a loopback address: 127.0.1.1; using 172.24.1.215 instead (on interface eth0)
22/03/01 22:41:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/01 22:41:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
import pandas as pd
from pyspark.sql import types

# q1

In [6]:
spark.version

'3.0.3'

# q2

In [7]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-03-01 22:43:47--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.140.81
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.140.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  4.62MB/s    in 2m 55s  

2022-03-01 22:46:43 (4.00 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [8]:
schema = types.StructType(
    [
        types.StructField('hvfhs_license_num', types.StringType(), True),
        types.StructField('dispatching_base_num', types.StringType(), True),
        types.StructField('pickup_datetime', types.TimestampType(), True),
        types.StructField('dropoff_datetime', types.TimestampType(), True),
        types.StructField('PULocationID', types.IntegerType(), True),
        types.StructField('DOLocationID', types.IntegerType(), True),
        types.StructField('SR_Flag', types.StringType(), True)
    ]
)

In [38]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [39]:
df.show()

+----------+-------+-------------------+-------------------+
|LocationID|Borough|               Zone|       service_zone|
+----------+-------+-------------------+-------------------+
|      null| B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|
|      null| B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|
|      null| B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|
|      null| B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|
|      null| B02872|2021-02-01 00:08:42|2021-02-01 00:17:57|
|      null| B02872|2021-02-01 00:26:02|2021-02-01 00:42:51|
|      null| B02872|2021-02-01 00:45:50|2021-02-01 01:02:50|
|      null| B02764|2021-02-01 00:06:42|2021-02-01 00:31:50|
|      null| B02764|2021-02-01 00:34:34|2021-02-01 00:58:13|
|      null| B02510|2021-02-01 00:03:43|2021-02-01 00:39:37|
|      null| B02510|2021-02-01 00:55:36|2021-02-01 01:08:39|
|      null| B02510|2021-02-01 00:06:13|2021-02-01 00:33:45|
|      null| B02510|2021-02-01 00:42:24|2021-02-01 01:11:31|
|      null| B02764|2021

22/03/01 23:22:50 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 7, schema size: 4
CSV file: file:///home/wtsang/projects/dataeng-zc/week5_batch_processing/code/fhvhv_tripdata_2021-02.csv


In [11]:
# format to parquet
df = df.repartition(24).write.parquet('fhvhv/2021/02/', mode='overwrite')

22/03/01 22:47:54 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 22:47:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 22:47:59 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 22:47:59 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [12]:
# folder size
!du -h fhvhv

208M	fhvhv/2021/02
208M	fhvhv/2021
208M	fhvhv


# q3 taxi trips

In [ ]:
from pyspark.sql import functions as F

In [40]:
df = spark.read.parquet('fhvhv/2021/02/*')
df.registerTempTable('fhvhv')

In [41]:
spark.sql("""
SELECT
    COUNT(*)
FROM
    fhvhv
WHERE
    DATE(pickup_datetime) = '2021-02-15'
""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



In [25]:
# q4 longest trip
spark.sql("""
SELECT
    pickup_datetime,
    unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime) as duration
FROM
    fhvhv
ORDER BY duration DESC
""").show()

+-------------------+--------+
|    pickup_datetime|duration|
+-------------------+--------+
|2021-02-11 13:40:44|   75540|
|2021-02-17 15:54:53|   57221|
|2021-02-20 12:08:15|   44039|
|2021-02-03 20:24:25|   40653|
|2021-02-19 23:17:44|   37577|
|2021-02-25 17:13:35|   35010|
|2021-02-20 01:36:13|   34806|
|2021-02-18 15:24:19|   34612|
|2021-02-18 01:31:20|   34555|
|2021-02-10 20:51:39|   34169|
|2021-02-10 01:56:17|   32476|
|2021-02-25 09:18:18|   32439|
|2021-02-21 19:59:13|   32223|
|2021-02-09 18:36:13|   32087|
|2021-02-06 09:48:09|   31447|
|2021-02-02 09:42:30|   30913|
|2021-02-10 10:12:08|   30856|
|2021-02-09 13:30:13|   30732|
|2021-02-21 22:50:52|   30660|
|2021-02-05 21:32:33|   30511|
+-------------------+--------+
only showing top 20 rows



In [32]:
#q5
from pyspark.sql.functions import desc

df.groupby('dispatching_base_num').count().sort(desc('count')).show()

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
|              B02869| 429720|
|              B02887| 322331|
|              B02871| 312364|
|              B02864| 311603|
|              B02866| 311089|
|              B02878| 305185|
|              B02682| 303255|
|              B02617| 274510|
|              B02883| 251617|
|              B02884| 244963|
|              B02882| 232173|
|              B02876| 215693|
|              B02879| 210137|
|              B02867| 200530|
|              B02877| 198938|
+--------------------+-------+
only showing top 20 rows



In [33]:
#6 common pair
schema = types.StructType(
    [
        types.StructField('LocationID', types.IntegerType(), True),
        types.StructField('Borough', types.StringType(), True),
        types.StructField('Zone', types.StringType(), True),
        types.StructField('service_zone', types.StringType(), True)
    ]
)

df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('taxi+_zone_lookup.csv')

df = df.repartition(24).write.parquet('zones/', mode='overwrite')

In [42]:
df_zones = spark.read.parquet('zones/')
df_zones.registerTempTable('zones')



In [43]:
spark.sql("""
SELECT
    CONCAT(coalesce(pu.Zone, 'Unknown'), '/', coalesce(do.Zone, 'Unknown')) AS pair,
    COUNT(*)
FROM
    fhvhv
LEFT JOIN zones as pu
    ON fhvhv.PULocationID = pu.LocationID
LEFT JOIN zones as do
    ON fhvhv.DOLocationID = do.LocationID
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT
    1
;
""").show()

+--------------------+--------+
|                pair|count(1)|
+--------------------+--------+
|East New York/Eas...|   45041|
+--------------------+--------+



22/03/02 01:00:43 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1547854 ms exceeds timeout 120000 ms
22/03/02 01:00:43 WARN SparkContext: Killing executors is not supported by current scheduler.
